# Week 3 - Second Year Project

---

**Learning goals**
* Understand the logic behind emission and transition probabilities
* Implement a POS tagger (or other sequence labeling tasks) using Viterbi
* Understand how to prepare textual data in a format usable for neural network model.
* Understand (and be able to implement) the forward pass of a feedforward neural network

**Notebook overview**

*Lecture 5*

1. Emission and transition probabilities: Calculate the emission and transition probabilities for the Viterbi algorithm
2. Viterbi algorithm: Implement the Viterbi algorithm, including the backtracking

*Lecture 6*

3. Representing the data: Convert input data to make it usable in neural networks
4. Forward pass (from scratch): Implement the forward pass as matrix multiplications
5. Loading existing weights: Check if your forward pass matches the forward pass as implemented in torch with pre-trained weights.

**Week 3 Competition: build POS tagger**

We have prepared two competitions that only differ in language of datasets, feel free to join either of the them:
- 🇩🇰 [Danish](https://www.kaggle.com/competitions/pos-tagging-da/overview)
- 🇬🇧 [English](https://www.kaggle.com/competitions/pos-tagging-en/overview)

The deadline for the submission of your solution to the competition is Sunday 19th of February (midnight). Good luck!

# Lecture 5: Sequence Prediction with HMMs

In this exercise you will implement the Viterbi algorithm for decoding in sequence tagging.

## 1. Emissions and transition probabilities

In this part of the exercise you are going to prepare the emission and transition probabilities to use in the viterbi algorithm. We are going to focus on the task of Parts-Of-Speech (POS) tagging. We will use the data that we annotated last week. You can use the following datareader for the following assignments:


In [1]:
import codecs

def read_conll_file(file_name):
    """
    read in conll file
    
    :param file_name: path to read from
    :yields: list of words and labels for each sentence
    """
    current_words = []
    current_tags = []

    for line in codecs.open(file_name, encoding='utf-8'):
        line = line.strip()

        if line:
            if line[0] == '#':
                continue # skip comments
            tok = line.split('\t')
            word = tok[0]
            tag = tok[1]

            current_words.append(word)
            current_tags.append(tag)
        else:
            if current_words:  # skip empty lines
                yield((current_words, current_tags))
            current_words = []
            current_tags = []

    # check for last one
    if current_tags != []:
        yield((current_words, current_tags))

label_dict = set()
for words, labels in read_conll_file('pos-data/train-students-da.conll'):
    for label in labels:
        label_dict.add(label)

SMOOTH = 0.1
UNK = '<UNK>'
BEG = '<S>'
END = '</S>'

label_dict.add(BEG)
label_dict.add(END)
# put labels in a list, so that they are guaranteed to have the same order
label_list = list(sorted(label_dict))
print(label_list)


['</S>', '<S>', 'ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X']


* a) calculate the emission probabilities based on the training data (`pos-data/train-students-da.conll` or `pos-data/train-students-en.conll` for English). Make sure that every POS tag can be assigned to an `<UNK>` token, use laplace smoothing with a value of 0.1 to avoid probabilities of 0.0.

**Hint**: The emission probability $P(w_i|t_{i})$ is the probability that given a tag, $t_i$, that it will be associated with a given word $w_i$. The formula below shows counts $C$ needed to calculate the probability.

$$P(w_i|t_{i}) = {C(t_{i},w_{i}) \over C(t_{i})}$$

* b) calculate the transition probabilities based on the training data, use a special label for the beginning of a sentence (`<S>`) and the end of a sentence (`</S>`). use laplace smoothing with a value of 0.1 to avoid probabilities of 0.0. 

**Hint**: The transition probability $P(t_i|t_{i-1})$ is the probability that given a tag, $t_{i-1}$, that it will be followed by a tag $t_i$.
$$P(t_i|t_{i-1}) = {C(t_{i-1},t_{i}) \over C(t_{i-1})}$$


You can check whether your solution is correct by estimating the probabilities on the Danish data and check whether the probabilities match:

In [ ]:
print(transition_probs['ADJ']['NOUN']) # 0.24911242603550302
print(transition_probs['NOUN']['ADJ']) # 0.013993174061433446
print(emission_probs['ADV']['ikke'])   # 0.08268025078369927
print(emission_probs['PART']['ikke'])  # 0.20608108108108097

## 2. Viterbi algorithm

In the image below we see an example of the calculation of the first 2 positions in a Viterbi decoding:
<img width=500px src="pics/viterbi.jpg">

* a) Implement Viterbi decoding, use the transition and emission probabilities previously defined. You can use the example code shown below as a starting point if you like.

**Hint**: The implementation can become simpler if you think about the problem as a 2d matrix that needs to be filled (each position in the list is a node in the viterbi decoding, $v_1(7)$, $v_1(6)$, ...). You can first initialize the matrix with 0.0's, and then fill it from left to right.

* b) Ensure that the best path is saved during the decoding, so that you can extract the labels. What is the accuracy of your implementation of the Viterbi algorithm on the development data (`pos-data/da_arto-dev.conll`)?

**Hint**: If implemented correctly, it should score at least an accuracy of 50%. If you score lower, we suggest you try printing the probabilities at each step (word) for the first sentence of the development data. You can also easily check in your transition probabilities whether the order `ADJ-NOUN` is much more probable as `NOUN-ADJ`, and for the emission probabilities you can check the probabilities per tag for a common word (like `hvor`, which should be `ADV` in almost all cases).

* c) Upload predictions of your final model on the test data (`pos-data/da_arto-test.conll`) on Kaggle. *Bonus: try to improve your predictions by inspecting common errors, or by tuning some of the decisions you made.*


In [ ]:
import pickle
emission_probs, transition_probs = pickle.load(open('probs_da.pickle', 'rb'))
print([x for x in transition_probs])
print(transition_probs['<S>'])


In [ ]:
# You can also load the pre-calculate probabilities:
# import pickle
# emission_probs, transition_probs = pickle.load(open('probs_da.pickle', 'rb'))
# emission_probs, transition_probs = pickle.load(open('probs_en.pickle', 'rb'))

labels = ['</S>', '<S>', 'ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X']
BEG = '<S>'
END = '</S>'
UNK = '<UNK>'
# This code assumes that you have a dictionary emission_probs which
# has a dictionary for every label which contains all words and UNK 
# with corresponding probabilities, e.g.:
# emission_probs['ADV']['ikke']
# 0.08268025078369927

# It also assumes you have a dictionary transition_probs which has 
# a dictionary for every label which contains outgoing labels and 
# corresponding probabilities, e.g.:
# transition_probs['ADJ']['NOUN']
# 0.24911242603550302
# transition_probs['NOUN']['ADJ']
# 0.013993174061433446

def viterbi(sentence):
    """
    sentence: list of strings
    """
    columns = len(sentence)
    rows = len(labels)
    
    # Create the full matrix for scores as well as the backtracking 
    # Note that there is space for  BEG and END labels included, but 
    # they do not have to be filled as they are never part of the 
    # final sequence. 
    # Note: If you use negative log probabilities, you would want to 
    # fill this with a very high number.
    # scores[0][3] should get the probability of the best path of the 
    # first word (this) and the 4th label (ADP)
    scores = [[0.0 for _ in range(columns)] for _ in range(rows)]
    backtrack = [[0 for _ in range(columns)] for _ in range(rows)]
    
    # Handle the first token separately, as it only has 2 probabilities (emission, transition)
    for pos_tag_idx, pos_tag in enumerate(labels):
        # TODO: make sure emission_probs is in the right format, or change the call
        # The probability of the first word given the POS tag:
        word = sentence[0]
        if word not in emission_probs[pos_tag]:
            word = UNK
        em_prob = emission_probs[pos_tag][word] 
        
        # TODO: make sure transition_probs is in the right format, or change the call
        # The probability of the POS tag given that the previous "tag" is <S>
        transition_prob = transition_probs[BEG][pos_tag]
        scores[pos_tag_idx][0] = em_prob * transition_prob
        
        # Backtracking is always the special token, because at position -1 there is
        # no other possibility
        backtrack[pos_tag_idx][0] = labels.index('<S>')
    
    # Now handle the rest of the sequence
    for word_pos in range(1, columns):
        for pos_tag_idx, pos_tag in enumerate(labels):
            if pos_tag in [BEG,END]:
                continue

            # Get emission probability
            # TODO
            
            # Get the probabilities for each possible path (including transition probability and history)
            # TODO
            
            # Now extract the best score and its previous path and save these
            # TODO

    # Handle the last special token, this only has a transition probability and a history probability
    # TODO

    # Extract the best score, and the best last tag
    # TODO

    # Extract the sequence by starting at the best_last_tag
    # TODO

#viterbi(['det', 'er', 'en', 'meget', 'god', 'chokolade', '.'])
#viterbi(['this', 'is', 'a', 'very', 'good', 'chocolate', '.'])

# Lecture 6 - Language Identification with a Feedforward Neural Network


In this exercise, you will implement the forward step of a FFNN from scratch and compare your solution to Pytorch on a small toy example to predict the language for a given word. 

It is very important that you understand the basic building blocks (input/output: how to encode your instances, the labels; the model: what the neural network consists of, how to learn its weights, how to do a forward pass for prediction). 

What you will learn:

* how to encode the data as character-based features and feed this n-hot representation as input to a FNN
* how to define the model (FNN) by reading off its structure from a graphical illustration of the network 
* compute the forward pass manually by loading existing weights for the model; to know whether your implementation is correct, you will compare the computed prediction scores to a model implemented in Pytorch



##  3. Representing the data

We are assuming multi-class classification tasks. The labels are $$ y \in \{da,nl,en\}$$

We will use the same data as in week2, from:
* English [Wookipedia](https://starwars.fandom.com/wiki/Main_Page)  
* Danish [Kraftens Arkiver](https://starwars.fandom.com/da/wiki) 
* Dutch [Yodapedia](https://starwars.fandom.com/da/wiki)


In [ ]:
def load_langid(path):
    text = []
    labels = []
    for line in open(path, encoding="utf-8"):
        tok = line.strip().split('\t')
        labels.append(tok[0])
        text.append(tok[1])
    return text, labels

wooki_train_text, wooki_train_labels = load_langid('langid-data/wookipedia_langid.train.tok.txt')


* a): Convert the training data into n-hot format, where each feature represents whether a single character is present or not.  Similarly, convert the labels into numeric format. For simplicity, you can assume a closed vocabulary (only the letters in wookie_train_text, no unknown-word handling). Keep original casing, and assign the character indices based on their chronological order.

  * What is the vocabulary size?
  
**Hint:** It is easier for the rest of the assignment if you directly use a torch tensor to save the features ([tutorial](https://pytorch.org/tutorials/beginner/blitz/tensor_tutorial.html#sphx-glr-beginner-blitz-tensor-tutorial-py), another [introduction](https://towardsdatascience.com/an-easy-introduction-to-pytorch-for-neural-networks-3ea08516bff2)), a 2d torch tensor filled with 0's can be initiated with: `torch.zeros(dim1, dim2, dtype=float)`. Note the use of `float` instead of `int` here, which is only because the `torch.mm` requires float tensors as input.

##  4: Forward pass (from scratch)

### Feedforward Neural Networks (FNNs) or MLPs

Feedforward Neural Networks (FNNs) are also called Multilayer Perceptrons (MLPs). These are the most basic types of neural networks. They are called this way as the information is flowing from the input nodes through the network up to the output nodes. 

It is essential to understand that a neural network is a non-linear classification model which is based upon function application. Each layer in a neural network is an application of a function.

Summary (by J.Frellsen):
<img src="pics/fnn_jf.png">

You are going to implement the forward step manually on a small dataset. You will create a network following the design in the following figure (note that the input should be the sames size as the number of characters found in the previous assignment, instead of 4):

<img src="pics/nn.svg">

a) How many neurons do hidden layer 1 and hidden layer 2 have? Note: the bias node is not shown in the figure, you do not have to count them for this assignment.

b) How many neurons does the output layer have? And the input layer? (Note: the figure shows only 4 input nodes, in this example your input size is defined in the previous assignment - what is the input layer size?)

c) Specify the size of layers of the feedforward neural network:

In [ ]:
## helper functions to determine the input and output dimensions of each layer
input_dim = 

hidden_dim1 = 
hidden_dim2 = 

output_dim = 

d) Now initialize the layers themselves as torch tensors (do not use a torch.module here!). You can define the bias and the weights in separate tensors. The weights should be initialized randomly (`torch.randn((dim1, dim2), dtype=torch.float)`, see also [torch.randn](https://pytorch.org/docs/stable/generated/torch.randn.html)) and the biases can be set to 1 (`torch.ones(dim1, dtype=torch.float)`, see also [torch.ones](https://pytorch.org/docs/stable/generated/torch.ones.html)). Confirm whether their size match the answer to `b)` and `a)` by printing .shape of the tensors.


In [ ]:
## define all parameters of this NN


Now that we have defined the shape of all parameters, we are ready to "connect the dots" and build the network. 

It is instructive to break the computation of each layer down into two steps: the scores $a1$ are obtained by the linear function followed by the activation applications $\sigma$ to obtain the representation $z1$, as in:

$$ a1 = xW_1 + b_1$$
$$ z1 = \sigma(a1)$$

d) Specify the entire network up to the output layer $z3$, and **up to and exclusive** the final application of the softmax, the last activation function, which is provided. For multiplication [torch.mm](https://pytorch.org/docs/stable/generated/torch.mm.html) can be used. Use a tanh activation function: [torch.tanh](https://pytorch.org/docs/stable/generated/torch.tanh.html).

The exact implementation of the softmax might differ from toolkit to toolkit (due to variations in implementation details in order to obtain numerical stability). Therefore, we will use the Pytorch implementation for the softmax calculation ([torch.nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html)).

In [ ]:
## implement the forward pass (up to and exclusive the softmax) 
## apply it to the training data `data_train` - use vectorization


We can check that all predictions sum up to approximately 1 (hint: use `torch.sum` with `axis=1`)




Congrats! you have made it through the manual construction of the forward pass. Note that these weights are still random, so performance is not expected to be good. Now lets compare your implementation to a set of pre-determined weights.

##  5. Where do the weights come from?  Loading existing weights

So far, the model that you used randomly initialized weights. In this step we will load pre-trained model weights and do the forward pass with those weights, in order to check your implementation against model predictions computed by the toolkit.

Now we are going to:
* load pretrained weights for all parameters
* apply the weights to the evaluation data
* check that your manual softmax scores match the ones obtained by the pre-trained model `model` that we will load
* convert the output to labels and calculate the accuracy score

First, lets load the pre-trained model:

In [ ]:
import torch
import torch.nn as nn

# use the character indexing from assignment 3
idx2char = ['H', 'e', ' ', 'v', 'n', 'w', 't', 's', 'o', 'f', 'a', 'r', 'u', 'g', 'h', ',', 'i', 'c', 'y', 'd', 'b', 'm', 'p', 'l', 'k', '.', 'D', 'E', 'C', 'j', 'R', 'S', 'U', '1', "'", 'æ', 'å', 'q', '`', 'I', '(', ')', 'M', 'F', '-', 'x', 'K', '9', '5', 'B', 'W', 'z', 'G', 'P', 'L', '/', 'O', '6', 'T', '7', 'Z', '2', '0', 'J', 'V', 'A', 'ø', 'X', '–', 'N', 'ë', ':', '&', '3', 'Y', 'é', '4', '[', ']', '’', ';', '8', 'É', 'Æ', 'Q', '!', '—', 'ï', '°', 'ō', '\u200b', '‘', 'ń', '“', '”', '?', 'Å', '<', '>', '#', '%', '+', 'ʊ', 'ɹ', 'ə', 'ɑ', 'ö', 'à', 'á', 'è', '=', 'ü', 'Ø', '∑', '^', 'ś', 'ñ', '|', '½', '$', '«', '™', 'ó', '´', '…', '―', '»', 'ː', 'θ', '²', 'Θ']
char2idx = {'H': 0, 'e': 1, ' ': 2, 'v': 3, 'n': 4, 'w': 5, 't': 6, 's': 7, 'o': 8, 'f': 9, 'a': 10, 'r': 11, 'u': 12, 'g': 13, 'h': 14, ',': 15, 'i': 16, 'c': 17, 'y': 18, 'd': 19, 'b': 20, 'm': 21, 'p': 22, 'l': 23, 'k': 24, '.': 25, 'D': 26, 'E': 27, 'C': 28, 'j': 29, 'R': 30, 'S': 31, 'U': 32, '1': 33, "'": 34, 'æ': 35, 'å': 36, 'q': 37, '`': 38, 'I': 39, '(': 40, ')': 41, 'M': 42, 'F': 43, '-': 44, 'x': 45, 'K': 46, '9': 47, '5': 48, 'B': 49, 'W': 50, 'z': 51, 'G': 52, 'P': 53, 'L': 54, '/': 55, 'O': 56, '6': 57, 'T': 58, '7': 59, 'Z': 60, '2': 61, '0': 62, 'J': 63, 'V': 64, 'A': 65, 'ø': 66, 'X': 67, '–': 68, 'N': 69, 'ë': 70, ':': 71, '&': 72, '3': 73, 'Y': 74, 'é': 75, '4': 76, '[': 77, ']': 78, '’': 79, ';': 80, '8': 81, 'É': 82, 'Æ': 83, 'Q': 84, '!': 85, '—': 86, 'ï': 87, '°': 88, 'ō': 89, '\u200b': 90, '‘': 91, 'ń': 92, '“': 93, '”': 94, '?': 95, 'Å': 96, '<': 97, '>': 98, '#': 99, '%': 100, '+': 101, 'ʊ': 102, 'ɹ': 103, 'ə': 104, 'ɑ': 105, 'ö': 106, 'à': 107, 'á': 108, 'è': 109, '=': 110, 'ü': 111, 'Ø': 112, '∑': 113, '^': 114, 'ś': 115, 'ñ': 116, '|': 117, '½': 118, '$': 119, '«': 120, '™': 121, 'ó': 122, '´': 123, '…': 124, '―': 125, '»': 126, 'ː': 127, 'θ': 128, '²': 129, 'Θ': 130}

# the label indexes that were used during training
label2idx = {'da':0, 'nl':1, 'en':2}
idx2label = ['da', 'nl', 'en']

# This is the definition of an FNN model in PyTorch, and can mostly be ignored for now.
# We will focus on how to create Torch models in week 5
class LangId(nn.Module):
    def __init__(self, vocab_size):
        super(Net, self).__init__()
        self.input = nn.Linear(vocab_size, 15)
        self.hidden1 = nn.Linear(15, 20)
        self.hidden2 = nn.Linear(20, 3)

    def forward(self, x):
        x = torch.tanh(self.input(x))
        x = torch.tanh(self.hidden1(x))
        x = self.hidden2(x)
        return x

lang_classifier = torch.load('model.th')


Inspect the weights you just loaded using the `state_dict()` function of the model: 

In [ ]:
lang_classifier.state_dict()


* a) Convert the following dev data into the input format for the neural network above. 

**Hint** The indices of the characters are based on the order in the training data, and should match in the development data, we provide the correct idx2char and char2idx that were used to train the model in the code above.

In [ ]:
wooki_dev_text, wooki_dev_labels = load_langid('langid-data/wookipedia_langid.dev.tok.txt')

* b) run a forward pass on the dev-data with `lang_classifier`, using the forward() function

* c) Apply your manual implementation of the forward pass to the evaluation data by using the parameters (weights) you just loaded with `state_dict()`. This allows you to check if you get the same results back as the model implemented in Torch. If the outputs match, you implemented the forward pass correctly, congratulations!

**Hint**: internally the torch model saves the weight in a transposed vector for efficiency reasons. This means that W1 will have the dimension of (15,131). To use your previous implementation you have to call the the transpose function in Pytorch ([`.t()`](https://pytorch.org/docs/stable/generated/torch.t.html)), which will convert the shape to be (131,15)

* d) Now apply softmax on the resulting weights and convert the output to the label predictions.

In [ ]:
# starting point for c)
W1 = lang_classifier.state_dict()['input.weight'].t()
